![Credit card being held in hand](credit_card.jpg)

Commercial banks receive _a lot_ of applications for credit cards. Many of them get rejected for many reasons, like high loan balances, low income levels, or too many inquiries on an individual's credit report, for example. Manually analyzing these applications is mundane, error-prone, and time-consuming (and time is money!). Luckily, this task can be automated with the power of machine learning and pretty much every commercial bank does so nowadays. In this workbook, you will build an automatic credit card approval predictor using machine learning techniques, just like real banks do.

### The Data

The data is a small subset of the Credit Card Approval dataset from the UCI Machine Learning Repository showing the credit card applications a bank receives. This dataset has been loaded as a `pandas` DataFrame called `cc_apps`. The last column in the dataset is the target value.

In [44]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import GridSearchCV

# Load the dataset
cc_apps = pd.read_csv("cc_approvals.data", header=None) 
cc_apps.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,b,30.83,0.000,u,g,w,v,1.25,t,t,1,g,0,+
1,a,58.67,4.460,u,g,q,h,3.04,t,t,6,g,560,+
2,a,24.50,0.500,u,g,q,h,1.50,t,f,0,g,824,+
3,b,27.83,1.540,u,g,w,v,3.75,t,t,5,g,3,+
4,b,20.17,5.625,u,g,w,v,1.71,t,f,0,s,0,+


In [45]:
cc_apps.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 690 entries, 0 to 689
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       690 non-null    object 
 1   1       690 non-null    object 
 2   2       690 non-null    float64
 3   3       690 non-null    object 
 4   4       690 non-null    object 
 5   5       690 non-null    object 
 6   6       690 non-null    object 
 7   7       690 non-null    float64
 8   8       690 non-null    object 
 9   9       690 non-null    object 
 10  10      690 non-null    int64  
 11  11      690 non-null    object 
 12  12      690 non-null    int64  
 13  13      690 non-null    object 
dtypes: float64(2), int64(2), object(10)
memory usage: 75.6+ KB


In [46]:
# cleaning data
clean_data = cc_apps.copy()

for col in clean_data.columns:
    clean_data[col] = clean_data[col].replace("?", np.NaN)
    if clean_data[col].dtype == "object":
        clean_data[col].fillna(clean_data[col].value_counts().index[0], inplace=True)
    else:
        clean_data[col].fillna(clean_data[col].mean(), inplace=True)
        
clean_data.iloc[:, 1] = clean_data.iloc[:, 1].astype("float")

In [47]:
# encoding 
clean_data_encoded = pd.get_dummies(clean_data, columns=[0, 3, 4, 5, 6, 8, 9, 11], drop_first=True)
X = clean_data_encoded.drop(13, axis=1).values
y = clean_data_encoded[13].values

# splitting the data 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# scaling the data 
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [48]:
# model development 

logit = LogisticRegression()
logit.fit(X_train_scaled, y_train)
y_pred = logit.predict(X_test_scaled)

# confusion matrix
conf_mat = confusion_matrix(y_test, y_pred)
acc_score = accuracy_score(y_test, y_pred)

print(conf_mat)
print(acc_score)

[[55 15]
 [11 57]]
0.8115942028985508


In [49]:
# hyperparameter tuning

params = {"C": [0.001, 0.01, 0.1, 1]}

grid_cv = GridSearchCV(logit, param_grid=params, cv=5, scoring="accuracy")
grid_cv.fit(X_train_scaled, y_train)
best_model = grid_cv.best_estimator_
best_params = grid_cv.best_params_
best_score = grid_cv.best_score_

print(f"Best params: {best_params}")
print(f"Best accuracy: {best_score}")

Best params: {'C': 0.1}
Best accuracy: 0.8714004914004914


In [50]:
best_model.fit(X_train_scaled, y_train)
y_pred_best = best_model.predict(X_test_scaled)
print(accuracy_score(y_test, y_pred_best))

0.8260869565217391
